In [2]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cmaps
from multiprocessing import Process
import time
import os
import matplotlib
import seaborn as sns
matplotlib.use('Agg')
from cnmaps import get_adm_maps, draw_maps
from scipy.stats import ttest_ind
import cmcrameri.cm as cmc
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from matplotlib import gridspec
import matplotlib
matplotlib.use('Agg')
from scipy.stats import genextreme
from tqdm import tqdm



In [3]:
def maskout(data,mask):
    #data is 3d, mask is 2d
    mask_useid=np.where(mask>0,1,np.nan)
    mask_useid=np.expand_dims(mask_useid,axis=0)
    mask_useid=np.repeat(mask_useid,data.shape[0],axis=0)
    data_maskout=data*mask_useid
    data_maskout[:,:,0:70]=np.nan
    return data_maskout

def mask2dout(data,mask):
    #data is 2d, mask is 2d
    mask_useid=np.where(mask>0,1,np.nan)
    data_maskout=data*mask_useid
    data_maskout[:,0:70]=np.nan
    return data_maskout

def cut_edge(data):
    #cut the lateral 7 grids
    #if the data is 3d, then cut the lateral 7 grids
    if len(data.shape)==3:
        data=data[:,7:-7,7:-7]
    #if the data is 2d, then cut the lateral 7 grids
    if len(data.shape)==2:
        data=data[7:-7,7:-7]
    return data

def cal_pcc_rmse(obs,model,mask,ifpr):
    mask[:,7:77]=-1
    mask=np.where(mask>0,1,np.nan)
    mask=np.where(np.isnan(obs),np.nan,mask)
    mask=np.where(np.isnan(model),np.nan,mask)

    obs=np.where(mask>0,obs,np.nan)
    model=np.where(mask>0,model,np.nan)
    obs1d=obs.flatten()
    model1d=model.flatten()

    # obs=np.where(mask>0,obs,np.nan)[:,7:77]
    # model=np.where(mask>0,model,np.nan)[:,7:77]
    # mask=np.where(mask>0,1,np.nan)[:,7:77]
    # obs1d=obs.flatten()
    # model1d=model.flatten()
    # mask1d=mask.flatten()

    # obs=np.where(np.abs(obs)>400,np.nan,obs)
    # model=np.where(np.abs(model)>400,np.nan,model)

    # obs1d=obs1d[np.where(mask1d>0)]
    # model1d=model1d[np.where(mask1d>0)]

    model1d=model1d[~np.isnan(obs1d)]
    obs1d=obs1d[~np.isnan(obs1d)]

    obs1d=obs1d[~np.isnan(model1d)]
    model1d=model1d[~np.isnan(model1d)]

    pcc=np.corrcoef(obs1d,model1d)[0,1]
    if ifpr:
        rmse=np.sqrt(np.nanmean(((model1d-obs1d)/obs1d)**2))
    else:
        rmse=np.sqrt(np.nanmean((obs1d-model1d)**2))
    return pcc,rmse



def seasmean(data):
    #data is 3d, 540 171 231; reshape to 45,12,171,231
    #the calculate the seasonal mean, DJF=0,1,11; MAM=2,3,4; JJA=5,6,7; SON=8,9,10
    data=np.reshape(data,(45,12,171,231))
    seasmean=np.ndarray(shape=(4,171,231))
    seasmean[0,:,:]=np.nanmean(data[:,[0,1,11],:,:],axis=(0,1))
    seasmean[1,:,:]=np.nanmean(data[:,[2,3,4],:,:],axis=(0,1))
    seasmean[2,:,:]=np.nanmean(data[:,[5,6,7],:,:],axis=(0,1))
    seasmean[3,:,:]=np.nanmean(data[:,[8,9,10],:,:],axis=(0,1))
    return seasmean


def extreme_PCT(data,pct):
    #data is 3d, 540 171 231
    #calculate the pct th percentile of the data
    pctdata=np.nanpercentile(data,pct,axis=0)
    return pctdata



def draw_map(lon,lat,matrix,modelname,varname,iseas,diff_levs,cm,pcc,rmse,CNmaskraw,figl):
    if modelname == "CWRF_CESM":
        modelname="CESM_CWRF"
    if modelname == "CWRF_MPI":
        modelname="MPI_CWRF"
    seasname=["DJF","MAM","JJA","SON","ANN","EXT10","EXT100","EXT1000","PCT95","PCT05","PCT90","PCT10","PCT99","PCT01"]
    matrix=cut_edge(matrix)
    matrix=mask2dout(matrix,CNmask)
    fig=plt.figure(figsize=(5,5))
    proj = ccrs.PlateCarree()  # 创建坐标系
    cwrf_cnproj = ccrs.LambertConformal(central_longitude=110.0, central_latitude=35.17781, false_easting=0.0, false_northing=0.0,  standard_parallels = (30, 60), globe=None, cutoff=-30)
    ax = plt.subplot(1, 1,1, projection=cwrf_cnproj)
    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.3, zorder=99)
    ax.add_feature(cfeat.OCEAN, facecolor="white", zorder=10)
    draw_maps(get_adm_maps(level='国'),color='grey',zorder=99,linewidth=0.8)
    draw_maps(get_adm_maps(level='省'),color='grey',zorder=99,linewidth=0.4)
    ax.set_extent([100,133,17.2,54])
    cn=ax.pcolormesh(lon,lat,matrix,cmap=cm,vmin=diff_levs[0],vmax=diff_levs[-1],transform = ccrs.PlateCarree())
    CNmaskraw_local=cut_edge(CNmaskraw)
    CNmaskraw_local=np.where(CNmaskraw_local==-1,-1,np.nan)
    ax.contourf(lon,lat,CNmaskraw_local,levels=[-1,0],colors='grey',transform = ccrs.PlateCarree(),zorder=100)
    if iseas<=3:
        ax.text(0.01,0.99,figl+") ", transform=ax.transAxes,color='k', ha='left', va='top',fontweight="bold", fontsize=14)
        if modelname in ["CESM","MPI"]:
            ax.text(0.5,1.02,seasname[iseas],fontweight="bold",transform=ax.transAxes,color='k', ha='center', va='bottom', fontsize=20)
        
        if seasname[iseas] == "DJF":
            ax.text(-0.02,0.5,modelname,fontweight="bold",transform=ax.transAxes,color='k', ha='right', va='center', fontsize=20,rotation=90)
    else:
        ax.text(0.01,0.99,figl+") "+modelname, transform=ax.transAxes,color='k', ha='left', va='top',fontweight="bold", fontsize=14)

    if not iseas in [8,9,10,11,12,13]:
        if varname=="PRAVG":
            ax.text(0.99,0.02,"PCC: "+str(round(pcc,2))+"\n"+"RRMSE: "+str(round(rmse,2)),
                    fontweight="bold",transform=ax.transAxes,color='k', ha='right', va='bottom', fontsize=12,zorder=1000)
        else:
            ax.text(0.99,0.02,"PCC: "+str(round(pcc,2))+"\n"+"RMSE: "+str(round(rmse,2)),
                    fontweight="bold",transform=ax.transAxes,color='k', ha='right', va='bottom', fontsize=12,zorder=1000)
    if modelname=="CESM" and iseas in [5,6,7]:
        ax.text(-0.03,0.5,"Returning Period: "+seasname[iseas].replace('EXT','')+"y",transform=ax.transAxes,color='k', ha='right', va='center', fontweight="bold", fontsize=16,rotation=90)
    plt.savefig('./pngs/'+modelname+"_"+seasname[iseas]+'_'+varname+'.png',dpi=300,bbox_inches='tight')
    os.system("convert -trim ./pngs/"+modelname+"_"+seasname[iseas]+'_'+varname+'.png ./pngs/'+modelname+"_"+seasname[iseas]+'_'+varname+'.png')

    if modelname=="OBS" and iseas>=8 and varname=="AT2M":
        fig=plt.figure(figsize=(1,6))
        cbar_ax = fig.add_axes([0.1, 0.07, 0.2, 0.85])
        cbar=plt.colorbar(cn,cax=cbar_ax,label="",orientation='vertical',shrink=0.02,extend='both')
        if iseas==8:
            cbar.ax.set_title("T2M\nPCT95:°C",fontsize=12)
        elif iseas==10:
            cbar.ax.set_title("T2M\nPCT90:°C",fontsize=12)
        elif iseas==12:
            cbar.ax.set_title("T2M\nPCT99:°C",fontsize=12)
        cbar.set_ticks(diff_levs[::2])
        cbar.ax.tick_params(labelsize=20)
        plt.savefig("./pngs/OBS"+varname+"_"+seasname[iseas]+'diffcolorbar_'+'.png',bbox_inches='tight')

    if modelname == 'CESM' and iseas == 4:
        fig=plt.figure(figsize=(12,2),dpi=300)
        cbar_ax = fig.add_axes([0.1, 0.07, 0.85, 0.06])
        if varname=="PRAVG":
            cbar=plt.colorbar(cn,cax=cbar_ax,label="PR Bias:mm/day",orientation='horizontal',shrink=0.02,extend='both')
            cbar.set_label("PR Bias:mm/day",fontsize=10,fontweight="bold")
        else:
            cbar=plt.colorbar(cn,cax=cbar_ax,label="T2M Bias:°C",orientation='horizontal',shrink=0.02,extend='both')
            cbar.set_label("T2M Bias:°C",fontsize=10,fontweight="bold")
        cbar.set_ticks(diff_levs[:])
        cbar.ax.tick_params(labelsize=10)
        plt.savefig("./pngs/"+varname+'_ANN_diffcolorbar_'+'.png',bbox_inches='tight')


    if modelname=="CESM" and iseas in [0,5,8,9,10,11,12,13]:
        fig=plt.figure(figsize=(1,20))
        cbar_ax = fig.add_axes([0.1, 0.07, 0.2, 0.85])
        if varname=="PRAVG":
            cbar=plt.colorbar(cn,cax=cbar_ax,label="PR :mm/day",orientation='vertical',shrink=0.02,extend='both')
            if iseas in [8,9]:
                cbar.set_label("PR-PCT05:mm/day",fontsize=22,fontweight="bold")
            elif iseas in [10,11]:
                cbar.set_label("PR-PCT10:mm/day",fontsize=22,fontweight="bold")
            elif iseas in [12,13]:
                cbar.set_label("PR-PCT01:mm/day",fontsize=22,fontweight="bold")
            else:
                cbar.set_label("PR :mm/day",fontsize=22,fontweight="bold")
        else:
            cbar=plt.colorbar(cn,cax=cbar_ax,label="T2M Bias:°C",orientation='vertical',shrink=0.02,extend='both')
            if iseas in [8,9]:
                cbar.set_label("T2M-PCT95 Bias:°C",fontsize=22,fontweight="bold")
            elif iseas in [10,11]:
                cbar.set_label("T2M-PCT90 Bias:°C",fontsize=22,fontweight="bold")
            elif iseas in [12,13]:
                cbar.set_label("T2M-PCT99 Bias:°C",fontsize=22,fontweight="bold")
            else:
                cbar.set_label("T2M Bias:°C",fontsize=22,fontweight="bold")
        cbar.set_ticks(diff_levs[:])
        cbar.ax.tick_params(labelsize=20)
        plt.savefig("./pngs/"+varname+"_"+seasname[iseas]+'diffcolorbar_'+'.png',bbox_inches='tight')
    
def gengevv_max(data,modelname,varname):
    shapeofdata = data.shape
    annual_maxima = np.max(data.reshape(-1, 12, shapeofdata[-2], shapeofdata[-1]), axis=1)
    # 定义返回期
    return_periods = [10, 100, 1000]
    results = np.zeros((len(return_periods), shapeofdata[-2], shapeofdata[-1]))
    # 遍历每个网格点，拟合GEV分布并计算阈值
    for i in tqdm(range(shapeofdata[-2])):
        for j in range(shapeofdata[-1]):
            if not np.isnan(annual_maxima[:, i, j]).any():
                maxima = annual_maxima[:, i, j]
                shape, loc, scale = genextreme.fit(maxima)
                for k, rp in enumerate(return_periods):
                    results[k, i, j] = genextreme.ppf(1 - 1/rp, shape, loc, scale)
    np.save(modelname+"_"+varname+"_hist_gev.npy",results)

def gengevv_min(data,modelname,varname):
    shapeofdata = data.shape
    annual_minima = np.min(data.reshape(-1, 12, shapeofdata[-2], shapeofdata[-1]), axis=1)
    # 定义返回期
    return_periods = [10, 100, 1000]
    results = np.zeros((len(return_periods), shapeofdata[-2], shapeofdata[-1]))
    # 遍历每个网格点，拟合GEV分布并计算阈值
    for i in tqdm(range(shapeofdata[-2])):
        for j in range(shapeofdata[-1]):
            if not np.isnan(annual_minima[:, i, j]).any():
                minima = annual_minima[:, i, j]
                minima_neg = -minima
                shape, loc, scale = genextreme.fit(minima_neg)
                for k, rp in enumerate(return_periods):
                    results[k, i, j] = -genextreme.ppf(1 - 1/rp, shape, loc, scale)
    np.save(modelname+"_"+varname+"_hist_gev.npy",results)


In [4]:


CNregionfil=Dataset("./newmask2.nc")
CNmaskraw=CNregionfil.variables["reg_mask"][:]
CNmaskraw=np.where(CNmaskraw==7,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==8,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==9,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==12,-1,CNmaskraw)
CNregionfil=Dataset("./CN_Subregion_new.nc")
lat2d=np.array(CNregionfil.variables["latitude"][:])
lon2d=np.array(CNregionfil.variables["longitude"][:])
lat2d=cut_edge(lat2d)
lon2d=cut_edge(lon2d)
CNmask=cut_edge(CNmaskraw)

infilname="./Fused_OBS_AT2M_PRAVG.nc"
infil=Dataset(infilname)
OBS_AT2M_hist=np.array(infil.variables['AT2M'])
OBS_PRAVG_hist=np.array(infil.variables['PRAVG'])

infilname="./reaggdata.nc"
infil=Dataset(infilname)
CESM_AT2M_hist=np.array(infil.variables['CESM_AT2M_hist'])
CESM_PRAVG_hist=np.array(infil.variables['CESM_PRAVG_hist'])
MPI_AT2M_hist=np.array(infil.variables['MPI_AT2M_hist'])
MPI_PRAVG_hist=np.array(infil.variables['MPI_PRAVG_hist'])
CWRF_CESM_AT2M_hist=np.array(infil.variables['CWRF_CESM_AT2M_hist'])
CWRF_CESM_PRAVG_hist=np.array(infil.variables['CWRF_CESM_PRAVG_hist'])
CWRF_MPI_AT2M_hist=np.array(infil.variables['CWRF_MPI_AT2M_hist'])
CWRF_MPI_PRAVG_hist=np.array(infil.variables['CWRF_MPI_PRAVG_hist'])



CESM_PRAVG_hist=CESM_PRAVG_hist*86400.0
MPI_PRAVG_hist=MPI_PRAVG_hist*86400.0
CWRF_CESM_PRAVG_hist=CWRF_CESM_PRAVG_hist
CWRF_MPI_PRAVG_hist=CWRF_MPI_PRAVG_hist*86400.0

OBS_AT2M_hist=np.where(OBS_AT2M_hist>100000,np.nan,OBS_AT2M_hist)
OBS_PRAVG_hist=np.where(OBS_PRAVG_hist>100000,np.nan,OBS_PRAVG_hist)


In [5]:


def barplot_PCC_RMSE(OBS,CESM,MPI,CWRF_CESM,CWRF_MPI,varname):
    if len(OBS.shape)==3:
        OBS_clim=np.nanmean(OBS,axis=0)
        CESM_clim=np.nanmean(CESM,axis=0)
        MPI_clim=np.nanmean(MPI,axis=0)
        CWRF_CESM_clim=np.nanmean(CWRF_CESM,axis=0)
        CWRF_MPI_clim=np.nanmean(CWRF_MPI,axis=0)
    else:
        OBS_clim=OBS
        CESM_clim=CESM
        MPI_clim=MPI
        CWRF_CESM_clim=CWRF_CESM
        CWRF_MPI_clim=CWRF_MPI
    if varname in ["PRAVGPCT01","PRAVGPCT05","PRAVGPCT10"]:
        ifpr=True
    else:
        ifpr=False
    CESM_PCC,CESM_RMSE=cal_pcc_rmse(OBS_clim,CESM_clim,CNmaskraw,ifpr)
    MPI_PCC,MPI_RMSE=cal_pcc_rmse(OBS_clim,MPI_clim,CNmaskraw,ifpr)
    CWRF_CESM_PCC,CWRF_CESM_RMSE=cal_pcc_rmse(OBS_clim,CWRF_CESM_clim,CNmaskraw,ifpr)
    CWRF_MPI_PCC,CWRF_MPI_RMSE=cal_pcc_rmse(OBS_clim,CWRF_MPI_clim,CNmaskraw,ifpr)
    print(CESM_PCC,MPI_PCC,CWRF_CESM_PCC,CWRF_MPI_PCC)
    categories = ['CESM','CESM_CWRF','MPI','MPI_CWRF']
    values1 = [CESM_PCC,CWRF_CESM_PCC,MPI_PCC,CWRF_MPI_PCC]
    values2 = [CESM_RMSE,MPI_RMSE,CWRF_CESM_RMSE,CWRF_MPI_RMSE]
    fig=plt.figure(figsize=(6, 7.5))
    ax1,ax2 = fig.subplots(2, 1, sharex=True)
    ax2.bar(categories, values2, color=['pink','red','lightblue','blue'],width=0.6)
    ax1.bar(categories, values1, color=['pink','red','lightblue','blue'],width=0.6)

    if varname in ["PRAVGPCT01","PRAVGPCT05","PRAVGPCT10"]:
        ax1.text(-0.18,1.01,"h)", transform=ax1.transAxes,color='k', ha='right', va='bottom',fontweight="bold", fontsize=27)
    else:
        ax1.text(-0.18,1.01,"b)", transform=ax1.transAxes,color='k', ha='right', va='bottom',fontweight="bold", fontsize=27)
    if varname in ["PRAVGPCT01","PRAVGPCT05","PRAVGPCT10"]:
        if varname=="PRAVGPCT05":
            ax1ybase=0.3
        else:
            ax1ybase=0.4
        if varname=="PRAVGPCT01":
            ax1ybase=0
        
    else:
        if varname=="AT2MPCT99":
            ax1ybase=0.6
        else:
            ax1ybase=0.8
    ax1.set_ylim((ax1ybase,1))
    ax1.set_xlim((-0.5,3.5))
    ax1.tick_params(axis='y', labelsize=16)
    ax1.set_ylabel('PCC',fontsize=18,fontweight="bold")
    for a,b in zip(categories, values1):
        ax1.text(a, b, '%.2f' % b, ha='center', va= 'bottom',fontsize=16,fontweight="bold")
        ax1.text(a,ax1ybase+0.01, a, ha='center',color='white', va= 'bottom',fontsize=13,fontweight="bold",rotation=90)
    if varname in ["PRAVGPCT01","PRAVGPCT05","PRAVGPCT10"]:
        if varname=="PRAVGPCT01":
            ax2.set_ylim((0,1.5))
        else:
            if varname=="PRAVGPCT05":
                ax2.set_ylim((0,2.1))
            else:
                ax2.set_ylim((0,1.8))
        ax2.set_ylabel('Relative-RMSE',fontsize=18,fontweight="bold")
    else:
        ax2.set_ylim((0,2.8))
        ax2.set_ylabel('RMSE:°C',fontsize=18,fontweight="bold")
    ax2.set_xticklabels(['',  '','', ''],fontsize=15,fontweight="bold")
    ax2.tick_params(axis='y', labelsize=16)
    for a,b in zip(categories, values2):
        if b >3:
            ax2.text(a, 1.3, '%.2f' % b, ha='center', va= 'bottom',fontsize=16,fontweight="bold")
        else:
            ax2.text(a, b, '%.2f' % b, ha='center', va= 'bottom',fontsize=16,fontweight="bold")
        if varname in ["PRAVGPCT01","PRAVGPCT05","PRAVGPCT10"]:
            ax2.text(a,0.025, a, ha='center',color='white', va= 'bottom',fontsize=13,fontweight="bold",rotation=90)
        else:
            ax2.text(a,0.05, a, ha='center',color='white', va= 'bottom',fontsize=13,fontweight="bold",rotation=90)
    plt.tight_layout()
    plt.savefig("./pngs/"+varname+"_rmsebar.png",bbox_inches='tight')
    print("Saved "+varname+"_rmsebar.png")


In [6]:
figletters_t2m=['a','b','c','d']
figletters_pr=['e','f','g','h']
AT2M_levs=np.linspace(-3,3,13)
PRAVG_levs=np.linspace(-5,5,11)

CESM_CWRF_AT2M_hist = CWRF_CESM_AT2M_hist
MPI_CWRF_AT2M_hist = CWRF_MPI_AT2M_hist
CESM_CWRF_PRAVG_hist = CWRF_CESM_PRAVG_hist
MPI_CWRF_PRAVG_hist = CWRF_MPI_PRAVG_hist


AT2M_CESM_climate_bias = np.nanmean(CESM_AT2M_hist - OBS_AT2M_hist,0)
AT2M_MPI_climate_bias = np.nanmean(MPI_AT2M_hist - OBS_AT2M_hist,0)
AT2M_CESM_CWRF_climate_bias = np.nanmean(CWRF_CESM_AT2M_hist - OBS_AT2M_hist,0)
AT2M_MPI_CWRF_climate_bias = np.nanmean(CWRF_MPI_AT2M_hist - OBS_AT2M_hist,0)

PRAVG_CESM_climate_bias = np.nanmean(CESM_PRAVG_hist - OBS_PRAVG_hist,0)
PRAVG_MPI_climate_bias = np.nanmean(MPI_PRAVG_hist - OBS_PRAVG_hist,0)
PRAVG_CESM_CWRF_climate_bias = np.nanmean(CWRF_CESM_PRAVG_hist - OBS_PRAVG_hist,0)
PRAVG_MPI_CWRF_climate_bias = np.nanmean(CWRF_MPI_PRAVG_hist - OBS_PRAVG_hist,0)


for var in ['AT2M','PRAVG']:
    for mid, model in enumerate(['CESM','CESM_CWRF','MPI','MPI_CWRF']):
        matrix=eval(var+"_"+model+"_climate_bias")
        obs=np.nanmean(eval("OBS_"+var+"_hist"),axis=0)
        modelmat=np.nanmean(eval(model+"_"+var+"_hist"),axis=0)
        if var=="AT2M":
            pcc,rmse=cal_pcc_rmse(obs,modelmat,CNmaskraw,False)
            diff_levs=AT2M_levs
            figletters=figletters_t2m
            print(model,var,pcc,rmse)
        else:
            pcc,rmse=cal_pcc_rmse(obs,modelmat,CNmaskraw,True)
            diff_levs=PRAVG_levs
            figletters=figletters_pr
            print(model,var,pcc,rmse)
        job= Process(target=draw_map, args=(lon2d,lat2d,matrix,model,var,4,diff_levs,cmaps.BlueWhiteOrangeRed,pcc,rmse,CNmaskraw,figletters[mid]))
        job.start()
job.join()



CESM AT2M 0.987036597930629 1.6184016
CESM_CWRF AT2M 0.9936764597892533 0.781448
MPI AT2M 0.9817686949530076 1.275689
MPI_CWRF AT2M 0.9937598682066314 0.8271595
CESM PRAVG 0.8731975144896718 0.5679964
CESM_CWRF PRAVG 0.9319685293660909 0.25908133
MPI PRAVG 0.9033173871979867 0.22960529
MPI_CWRF PRAVG 0.930662830157803 0.25472856


In [7]:
!convert +append ./pngs/CESM_ANN_AT2M.png pngs/CESM_CWRF_ANN_AT2M.png pngs/MPI_ANN_AT2M.png pngs/MPI_CWRF_ANN_AT2M.png ./pngs/lineone.png
!convert -resize 4140x ./pngs/AT2M_ANN_diffcolorbar_.png ./pngs/lineonecolorbar.png
!convert -append ./pngs/lineone.png ./pngs/lineonecolorbar.png ./pngs/AT2M_ANN.png

!convert +append ./pngs/CESM_ANN_PRAVG.png pngs/CESM_CWRF_ANN_PRAVG.png pngs/MPI_ANN_PRAVG.png pngs/MPI_CWRF_ANN_PRAVG.png ./pngs/lineone.png
!convert -resize 4140x ./pngs/PRAVG_ANN_diffcolorbar_.png ./pngs/lineonecolorbar.png
!convert -append ./pngs/lineone.png ./pngs/lineonecolorbar.png ./pngs/PRAVG_ANN.png

!convert -append ./pngs/AT2M_ANN.png ./pngs/PRAVG_ANN.png ./pngs/ANN.png

In [21]:

ppcctt=10
figletters_t2m=['a','e','c','f','d']
figletters_pr=['g','k','i','l','j']
AT2M_levs=np.linspace(-5,5,11)
PRAVG_levs=np.linspace(-1,1,11)
OBS_AT2M_hist_pcthigh=extreme_PCT(OBS_AT2M_hist,100-ppcctt)
OBS_PRAVG_hist_pctlow=extreme_PCT(OBS_PRAVG_hist,ppcctt)
CESM_AT2M_hist_pcthigh=extreme_PCT(CESM_AT2M_hist,100-ppcctt)
CESM_PRAVG_hist_pctlow=extreme_PCT(CESM_PRAVG_hist,ppcctt)
MPI_AT2M_hist_pcthigh=extreme_PCT(MPI_AT2M_hist,100-ppcctt)
MPI_PRAVG_hist_pctlow=extreme_PCT(MPI_PRAVG_hist,ppcctt)
CESM_CWRF_AT2M_hist_pcthigh=extreme_PCT(CWRF_CESM_AT2M_hist,100-ppcctt)
CESM_CWRF_PRAVG_hist_pctlow=extreme_PCT(CWRF_CESM_PRAVG_hist,ppcctt)
MPI_CWRF_AT2M_hist_pcthigh=extreme_PCT(CWRF_MPI_AT2M_hist,100-ppcctt)
MPI_CWRF_PRAVG_hist_pctlow=extreme_PCT(CWRF_MPI_PRAVG_hist,ppcctt)
# model_name=['OBS','CESM','CWRF_CESM','MPI','CWRF_MPI']
# for mid, modelone in enumerate(model_name):
#     isea=4
#     figl=str(chr(isea+mid*4+97))
#     obs=OBS_AT2M_hist_pcthigh[:,:]
#     model=eval(modelone+"_AT2M_hist_pcthigh")[:,:]
#     matrix=model-obs
#     pcc,rmse=cal_pcc_rmse(obs,model,CNmaskraw,False)
#     cm=cmaps.BlueWhiteOrangeRed
#     job = Process(target=draw_map, args=(lon2d,lat2d,matrix,modelone,"AT2M",8,AT2M_levs,cm,pcc,rmse,CNmaskraw,figl))
#     # job.start()
#     obs=OBS_PRAVG_hist_pctlow[:,:]
#     model=eval(modelone+"_PRAVG_hist_pctlow")[:,:]
#     matrix=model-obs
#     pcc,rmse=cal_pcc_rmse(obs,model,CNmaskraw,True)
#     job = Process(target=draw_map, args=(lon2d,lat2d,matrix,modelone,"PRAVG",9,PRAVG_levs,cm,pcc,rmse,CNmaskraw,figl))
#     # job.start()



In [22]:
for vid, var in enumerate(["AT2M","PRAVG"]):
    for mid,modelname in enumerate(["OBS","CESM","CESM_CWRF","MPI","MPI_CWRF"]):
        PRAVG_levs=np.linspace(0,2,11)
        AT2M_levs=np.linspace(-5,5,11)
        #annual
        uselevs=eval(var+"_levs")
        if var=="PRAVG":
            cm=cmaps.WhiteBlueGreenYellowRed
            matrix=eval(modelname+"_"+var+"_hist_pctlow")
            obsmatrix=eval("OBS_"+var+"_hist_pctlow")
            pcc,rmse=cal_pcc_rmse(matrix,obsmatrix,CNmaskraw,False)
            figl=figletters_pr[mid]
        else:
            # cm=cmc.vik
            cm=cmaps.BlueWhiteOrangeRed
            obsmatrix=eval("OBS_"+var+"_hist_pcthigh")
            pcc,rmse=cal_pcc_rmse(eval(modelname+"_"+var+"_hist_pcthigh"),obsmatrix,CNmaskraw,True)
            matrix=eval(modelname+"_"+var+"_hist_pcthigh")-eval("OBS_"+var+"_hist_pcthigh")
            if var=="AT2M" and modelname=="OBS":
                matrix=eval(modelname+"_"+var+"_hist_pcthigh")-273.15
                cm=cmaps.WhiteBlueGreenYellowRed
                uselevs=np.linspace(10,35,26)
            figl=figletters_t2m[mid]
        job=Process(target=draw_map,args=(lon2d,lat2d,matrix,modelname,var,10+vid,uselevs,cm,pcc,rmse,CNmaskraw,figl))#9010
        # job=Process(target=draw_map,args=(lon2d,lat2d,matrix,modelname,var,8+vid,uselevs,cm,pcc,rmse,CNmaskraw,figl))#9505
        # job=Process(target=draw_map,args=(lon2d,lat2d,matrix,modelname,var,12+vid,uselevs,cm,pcc,rmse,CNmaskraw,figl))#9901
        job.start()
# job.join()



OBS=OBS_PRAVG_hist_pctlow
CESM=CESM_PRAVG_hist_pctlow
MPI=MPI_PRAVG_hist_pctlow
CESM_CWRF=CESM_CWRF_PRAVG_hist_pctlow
MPI_CWRF=MPI_CWRF_PRAVG_hist_pctlow
job=Process(target=barplot_PCC_RMSE,args=(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF,"PRAVGPCT10"))
# job=Process(target=barplot_PCC_RMSE,args=(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF,"PRAVGPCT05"))
# job=Process(target=barplot_PCC_RMSE,args=(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF,"PRAVGPCT01"))
job.start()

OBS=OBS_AT2M_hist_pcthigh
CESM=CESM_AT2M_hist_pcthigh
MPI=MPI_AT2M_hist_pcthigh
CESM_CWRF=CESM_CWRF_AT2M_hist_pcthigh
MPI_CWRF=MPI_CWRF_AT2M_hist_pcthigh
job=Process(target=barplot_PCC_RMSE,args=(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF,"AT2MPCT90"))
# job=Process(target=barplot_PCC_RMSE,args=(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF,"AT2MPCT95"))
# job=Process(target=barplot_PCC_RMSE,args=(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF,"AT2MPCT99"))
job.start()

job.join()



0.94059821132129770.7813223727219799  0.54307531666981630.8896790618898919 0.8960836262771604  0.97944689251966920.8400689362233934
 0.9709970968119496
Saved AT2MPCT90_rmsebar.png
Saved PRAVGPCT10_rmsebar.png

In [12]:
cm=cmaps.BlueWhiteOrangeRed
AT2M_levs=np.linspace(-10,10,11)
PRAVG_levs=np.linspace(-1,1,11)
for reti, returnp in enumerate(['10','100']):
     for mid, modelone in enumerate(['CESM','MPI','CWRF_CESM','CWRF_MPI']):
        figl=str(chr(reti*4+mid+97))
        model=np.load(modelone+"_AT2M_hist_gev.npy")[reti,:,:]
        obs=np.load("OBS_AT2M_hist_gev.npy")[reti,:,:]
        diff=model-obs
        model=np.where(np.abs(diff)>9,np.nan,model)
        obs=np.where(np.abs(diff)>9,np.nan,obs)
        pcc,rmse=cal_pcc_rmse(obs,model,CNmaskraw,False)
        matrix=model-obs
        print(np.nanmin(matrix))
        job = Process(target=draw_map, args=(lon2d,lat2d,matrix,modelone,"AT2M",5+reti,AT2M_levs,cm,pcc,rmse,CNmaskraw,figl))
        job.start()
        figl=str(chr(reti*4+mid+97+8))

        model=np.load(modelone+"_PRAVG_hist_gev.npy")[reti,:,:]
        obs=np.load("OBS_PRAVG_hist_gev.npy")[reti,:,:]
        diff=model-obs
        model=np.where(np.abs(diff)>10,np.nan,model)
        obs=np.where(np.abs(diff)>10,np.nan,obs)
        pcc,rmse=cal_pcc_rmse(obs,model,CNmaskraw,False)
        matrix=model-obs
        job = Process(target=draw_map, args=(lon2d,lat2d,matrix,modelone,"PRAVG",5+reti,PRAVG_levs,cm,pcc,rmse,CNmaskraw,figl))
        job.start()

job.join()


-3.884904553032527
-5.6421191349856485
-3.563659057054565
-4.509185995355381
-8.706378967380545
-5.5795675226194135
-8.022399473634096
-8.323114827580753


In [20]:
model=cut_edge(model)
print(CNmaskl.shape)
print(model.shape)
model=mask2dout(model,CNmaskl)
temp=model.flatten()
temp=temp[~np.isnan(temp)]
temp.shape

(157, 217)
(157, 217)


(6012,)